In [ ]:
import pandas as pd
import numpy as np
import random

In [ ]:
random.seed("eKichabi")

# Load Data

In [ ]:
# firm census for comparison
df_census = pd.read_csv("./census_data_trimmed.csv")
unique_census_numbers = pd.concat([df_census['mobile_number1'], df_census['mobile_number2']], ignore_index=True).dropna().astype('int64').unique()

In [ ]:
# firm whitelist assignments
df = pd.read_csv("./firm_visibility_assignments.csv")
df = df.astype({'mobile_number1': pd.Int64Dtype(), 'mobile_number2': pd.Int64Dtype()})

In [ ]:
# drop firms/rows with no phone number
df_nums = df[df["mobile_number1"].notna()]

In [ ]:
unique_numbers = pd.concat([df_nums['mobile_number1'], df_nums['mobile_number2']], ignore_index=True).dropna().astype('int64').unique()

## General Stats

In [ ]:
# number of firms
num_firms = df.shape[0]
num_firms

In [ ]:
# number of census firms
df_census.drop_duplicates().shape[0]

In [ ]:
# numbers in firm assignments not in census
[number for number in unique_numbers if number not in unique_census_numbers]

In [ ]:
# numbers in census not in firm assignments
[number for number in unique_census_numbers if number not in unique_numbers]

In [ ]:
# number of unique phone numbers
len(unique_numbers)

In [ ]:
# number of unique census numbers
len(unique_census_numbers)

In [ ]:
# number of firms without a phone number
num_no_num = df[df["mobile_number1"].isna()].shape[0]
num_no_num

In [ ]:
# number of firms to whitelist/assign
num_firms_to_whitelist = df.shape[0] - df[df["mobile_number1"].isna()].shape[0]
num_firms_to_whitelist

# Reassign Firms

In [ ]:
# find groups of numbers that appear together
appears_together = {}
for i, row in df_nums.iterrows():
    if not pd.isna(row["mobile_number2"]):
        n1 = row["mobile_number1"]
        n2 = row["mobile_number2"]
        if n1 not in appears_together.keys():
            appears_together[n1] = set()
        appears_together[n1].add(n2)
        if n2 not in appears_together.keys():
            appears_together[n2] = set()
        appears_together[n2].add(n1)

In [ ]:
# assign groups
group_assignments = {}

lucky_groups = set()

num_overlapping_groups = 0
for k, v in appears_together.items():
    choices = []
    choices += (list(df_nums[df_nums["mobile_number1"] == k]["owner_names_visible"]))
    for i in v:
        choices += (list(df_nums[df_nums["mobile_number1"] == i]["owner_names_visible"]))

    if len(choices) == 1:
        continue

    unique_choices = set(choices)
    if len(unique_choices) == 1:
        for i in v:
            lucky_groups.add(i)
        lucky_groups.add(k)
        continue
    
    num_overlapping_groups += 1
    selection = random.choice(list(unique_choices))
    for i in v:
        group_assignments[i] = selection
    group_assignments[k] = selection

In [ ]:
# fix group conflicts
conflicts_fixed = []
reason = []
for i, row in df_nums.iterrows():
    n1 = row["mobile_number1"]
    if n1 in group_assignments.keys():
        conflicts_fixed.append(group_assignments[n1])
        reason.append("group conflict")
    else:
        conflicts_fixed.append(pd.NA)
        reason.append("")

df_nums = df_nums.assign(conflicts_fixed = conflicts_fixed, reason =  reason)

In [ ]:
# annotate lucky groups
df_nums.loc[df_nums['mobile_number1'].isin(lucky_groups) | df_nums['mobile_number2'].isin(lucky_groups), "reason"] = "lucky group"

In [ ]:
# re-assign duplicates that are not part of any groups
for number in df_nums[df_nums.duplicated(subset="mobile_number1")]['mobile_number1']:
    if any(list(df_nums.loc[df_nums['mobile_number1'] == number,'reason'])):
        continue
    df_nums.loc[df_nums['mobile_number1'] == number, 'conflicts_fixed'] = random.choice(list(set(df_nums[df_nums['mobile_number1'] == number]['owner_names_visible'])))
    df_nums.loc[df_nums['mobile_number1'] == number, 'reason'] += ("duplicate phone #")

## Group reassignment stats

In [ ]:
# reassigned firms by reason
df_nums["reason"].value_counts()

In [ ]:
# number of groups of 3
num_groups_of_three = sum([1 for val in appears_together.values() if len(val) >= 2])
num_groups_of_three

In [ ]:
# number of groups
(len(appears_together) - num_groups_of_three) // 2

In [ ]:
# number of groups with conflicts
num_overlapping_groups

In [ ]:
# number of groups with potential conflicts
num_overlapping_groups + len(lucky_groups)

In [ ]:
# number of firms with potential conflict
gr = df_nums[(df_nums['reason'] == 'group conflict') | (df_nums['reason'] == 'lucky group')]
len(gr)

In [ ]:
# number of phone numbers with conflict or got lucky (i.e. potential conflict)
len(set(pd.concat([gr[~gr["mobile_number2"].isna()]["mobile_number2"], gr["mobile_number1"]])))

In [ ]:
# number of groups with exactly 3 numbers
groups_with_three_numbers = dict()
for k, v in appears_together.items():
    if len(v) < 2:
        continue
    groups_with_three_numbers[k] = v
len(groups_with_three_numbers)    

In [ ]:
# number of unique phone numbers from the groups with exactly 3 numbers
groups_with_three_numbers_phone_numbers = set()
for k, v in groups_with_three_numbers.items():
    groups_with_three_numbers_phone_numbers.add(k)
    for nb in v:
        groups_with_three_numbers_phone_numbers.add(nb)
len(groups_with_three_numbers_phone_numbers)

In [ ]:
# number of firms part of the groups of exactly 3 numbers
f = set()
for i in groups_with_three_numbers_phone_numbers:
    for t in df_nums[(df_nums["mobile_number1"] == i) | (df_nums["mobile_number2"] == i)]['firmID']:
        f.add(t)
len(f)

In [ ]:
# number of numbers in duplicated firms not part of any groups
df_nums[df_nums['reason'] == 'duplicate phone #']['mobile_number1'].nunique()

In [ ]:
# number of duplicated phone numbers
total_numbers = pd.concat([df_nums[~df_nums["mobile_number2"].isna()]["mobile_number2"], df_nums["mobile_number1"]])
len(total_numbers) - len(unique_numbers)

In [ ]:
# number of firms with duplicated numbers (including any reason)
duplicated_numbers = total_numbers[total_numbers.duplicated()]
len(df_nums[(df_nums["mobile_number1"].isin(duplicated_numbers) | df_nums["mobile_number2"].isin(duplicated_numbers))])

In [ ]:
# numbers that are duplicated but not because of a potential group conflict
len(df_nums[(~df_nums["reason"].isin(['group conflict', 'lucky group'])) & (df_nums["mobile_number1"].isin(duplicated_numbers) | df_nums["mobile_number2"].isin(duplicated_numbers))])

^ this matches the number we got by reassigning which is good

also, 917 - 808 = 109 potential group conflicts and we are accounting for all of them!

### Examples of reassignments

In [ ]:
# random example of group conflict (appears with 2 or 3 unique numbers any amount of times, or with one number but that number appears with another number)
groupConflicts = df_nums[df_nums["reason"] == "group conflict"]
random_number = random.choice(list(pd.concat([groupConflicts['mobile_number1'], groupConflicts['mobile_number2']]).dropna().unique()))
numbers_in_group = pd.Series(df_nums[(df_nums['mobile_number1'] == random_number) | (df_nums['mobile_number2'] == random_number)][['mobile_number1', 'mobile_number2']].values.ravel()).dropna()
df_nums[(df_nums['mobile_number1'].isin(numbers_in_group)) | (df_nums['mobile_number2'].isin(numbers_in_group))]

In [ ]:
# random example of a lucky group (could have a conflict but didn't)
luckies = df_nums[df_nums["reason"] == "lucky group"]
random_number = random.choice(list(pd.concat([luckies['mobile_number1'], luckies['mobile_number2']]).dropna().unique()))
numbers_in_group = pd.Series(df_nums[(df_nums['mobile_number1'] == random_number) | (df_nums['mobile_number2'] == random_number)][['mobile_number1', 'mobile_number2']].values.ravel()).dropna()
df_nums[(df_nums['mobile_number1'].isin(numbers_in_group)) | (df_nums['mobile_number2'].isin(numbers_in_group))]

In [ ]:
# random example of duplicate not in group
groupConflicts = df_nums[df_nums["reason"] == "duplicate phone #"]
random_number = random.choice(list(pd.concat([groupConflicts['mobile_number1'], groupConflicts['mobile_number2']]).dropna().unique()))
# should not be any other numbers than random_number:
numbers_in_group = pd.Series(df_nums[(df_nums['mobile_number1'] == random_number) | (df_nums['mobile_number2'] == random_number)][['mobile_number1', 'mobile_number2']].values.ravel()).dropna()
df_nums[(df_nums['mobile_number1'].isin(numbers_in_group)) | (df_nums['mobile_number2'].isin(numbers_in_group))]

In [ ]:
# groups with 3 numbers
groups_with_three_numbers

# Save Reassignments

In [ ]:
df_nums.to_csv('./firm_reassignments.csv', index=False)

In [ ]:
# turn into AB testing format and save
AB_formatted = df_nums.copy(True)
AB_formatted['conflicts_fixed'] = AB_formatted['conflicts_fixed'].fillna(df_nums['owner_names_visible'])
AB_formatted = AB_formatted.rename(columns={'conflicts_fixed': 'group'})
AB_formatted = AB_formatted[['mobile_number1', 'mobile_number2', 'group']].replace(to_replace={'group': {1: 'A', 0: 'B'}})
AB_formatted.to_csv('./AB_test_groups.csv', index=False)

In [ ]:
# create anonymous version of census_data_trimmed.csv
census = pd.read_csv('census_data_trimmed.csv')
census.head()

In [ ]:
census['ownername'] = 'mtu asiyejulikana'

In [ ]:
census.to_csv('census_data_anonymous.csv', index=False)